In [ ]:
import os
import re
import ast
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import pointbiserialr
from datasets import load_dataset, load_from_disk
from sklearn.metrics import classification_report

tqdm.pandas()
sns.set_style("whitegrid")
sns.set_context("notebook")
aug_regex = re.compile(r"<aug>(.*?)</aug>", re.DOTALL)

In [ ]:
# set seeds
np.random.seed(42)

In [ ]:
# plotting constants
TITLE_FONT_SIZE = 16
WSPACE = 0.3
FIGURE_HEIGHT = 4
LINE_WIDTH = 2
FIG_SIZE = 4.5

In [ ]:

inference_logs = load_from_disk("data/combined_dataset")
list(inference_logs.keys())

In [ ]:
def parse_task_name(split_name):
    return "Sentiment" if "Sentiment" in split_name else "Toxicity" if "Toxicity" in split_name else "News"

def parse_distribution(split_name):
    return split_name.split("_")[-3]

def parse_model(split_name):
    return split_name.split("_")[-2]

def parse_tta_method(split_name):
    return split_name.split("_")[-1]

## No TTA Baselines

In [ ]:
no_tta_accuracies = {}
for split_name in tqdm(inference_logs):
    split_frame = inference_logs[split_name].to_pandas()
    split_no_tta_accuracy = classification_report(split_frame["label"], split_frame["original_predicted_class"], output_dict=True)["accuracy"]
    no_tta_accuracies[split_name] = split_no_tta_accuracy

no_tta_accuracies

## Analyze Main Results

In [ ]:
main_results_bert_splits = [split for split in inference_logs.keys() if "Ablate" not in split]
datasets = ["BOSS_Sentiment", "BOSS_Toxicity", "AgNewsTweets"]
split_data = {}

for task_name in datasets:
    if task_name not in split_data:
        split_data[task_name] = {}

    for split in tqdm(main_results_bert_splits):
        if task_name in split:
            distribution = split.split("_")[-3]
            model = split.split("_")[-2]
            tta_method = split.split("_")[-1]
            baseline_accuracy = classification_report(inference_logs[split]["label"], inference_logs[split]["original_predicted_class"], output_dict=True)["accuracy"]
            tta_accuracy = classification_report(inference_logs[split]["label"], inference_logs[split]["tta_predicted_class"], output_dict=True)["accuracy"]
            accuracy_delta = tta_accuracy - baseline_accuracy

            if model not in split_data[task_name]:
                split_data[task_name][model] = {}

            if tta_method not in split_data[task_name][model]:
                split_data[task_name][model][tta_method] = {}

            split_data[task_name][model][tta_method][distribution] = {
                "distribution": distribution,
                "model": model,
                "tta_method": tta_method,
                "baseline_accuracy": baseline_accuracy,
                "tta_accuracy": tta_accuracy,
                "accuracy_delta": accuracy_delta
            }

    # get the Accuracy Gain for each method excluding ID
    for model_name in split_data[task_name]:
        for tta_method in split_data[task_name][model_name]:
            accuracy_deltas = []
            baseline_accuracies = []
            for distribution in split_data[task_name][model_name][tta_method]:
                if distribution == "ID":
                    continue

                current_tta_result = split_data[task_name][model_name][tta_method][distribution]
                accuracy_deltas.append(current_tta_result["accuracy_delta"])
                baseline_accuracies.append(current_tta_result["baseline_accuracy"])

            split_data[task_name][model_name][tta_method]["mean_accuracy_delta"] = np.mean(accuracy_deltas)
            split_data[task_name][model_name][tta_method]["mean_baseline_accuracy"] = np.mean(baseline_accuracies)

print(json.dumps(split_data, indent=4))

In [ ]:
# create dataframe where there is a column for dataset, model, id_accuracy_delta, and mean_accuracy_delta
records = []

for task_name in tqdm(split_data):
    for model_name in split_data[task_name]:
        for tta_method in split_data[task_name][model_name]:
            current_tta_result = split_data[task_name][model_name][tta_method]
            records.append({
                "dataset": task_name,
                "model": model_name,
                "tta_method": tta_method,
                "id_accuracy_delta": current_tta_result["ID"]["accuracy_delta"] * 100 if "ID" in current_tta_result else None,
                "ood_mean_accuracy_delta": current_tta_result["mean_accuracy_delta"] * 100,
                "ood_mean_baseline_accuracy": current_tta_result["mean_baseline_accuracy"] * 100,
            })


main_results_frame = pd.DataFrame(records)
for model in split_data[task_name]:
    display(model)
    display(main_results_frame[main_results_frame["model"] == model].drop(columns="model").groupby(["dataset", "tta_method"]).mean().T)


# Analyze Across Dataset Sizes

In [ ]:
all_splits = [split for split in inference_logs.keys() if "Ablate_Data" in split]
datasets = set([("_".join(split.split("_")[:2]).replace("_Ablate", "")) for split in all_splits if "Ablate_Data" in split])
results = {}
# print(datasets)

for task_name in datasets:
    results[task_name] = {}

    for split_name in tqdm(all_splits, desc=task_name):
        if task_name in split_name:
            data_count = int(split_name.split("_")[-2].replace("BERT", ""))
            tta_method = split_name.split("_")[-1]
            shift_name = split_name.split("_")[-3]
            # print(dataset, data_count, tta_method, shift_name)

            baseline_accuracy = classification_report(inference_logs[split_name]["label"], inference_logs[split_name]["original_predicted_class"], output_dict=True)["accuracy"]
            tta_accuracy = classification_report(inference_logs[split_name]["label"], inference_logs[split_name]["tta_predicted_class"], output_dict=True)["accuracy"]

            if task_name not in results:
                results[task_name] = {}
            if data_count not in results[task_name]:
                results[task_name][data_count] = {}
            if tta_method not in results[task_name][data_count]:
                results[task_name][data_count][tta_method] = {}
            if shift_name not in results[task_name][data_count][tta_method]:
                results[task_name][data_count][tta_method][shift_name] = {}

            results[task_name][data_count][tta_method][shift_name] = {
                "method": tta_method,
                "baseline_accuracy": baseline_accuracy,
                "tta_accuracy": tta_accuracy,
                "baseline_delta": tta_accuracy - baseline_accuracy,
            }

            # inference_frames[task_name] = inference_logs[split].to_pandas()
            # break

print(json.dumps(results, indent=4))

In [ ]:
method_avg_delta = {}
for task_name in results:
    for data_count in results[task_name]:
        tta_perf_deltas = {}
        for tta_method in results[task_name][data_count]:
            for ood_shift in results[task_name][data_count][tta_method]:
                if tta_method not in tta_perf_deltas:
                    tta_perf_deltas[tta_method] = []

                shift_method_perf_deta = results[task_name][data_count][tta_method][ood_shift]["baseline_delta"]
                tta_perf_deltas[tta_method].append(shift_method_perf_deta)

        if task_name not in method_avg_delta:
            method_avg_delta[task_name] = {}

        method_avg_delta[task_name][data_count] = { tta_method: np.mean(tta_perf_deltas[tta_method]) for tta_method in tta_perf_deltas }
        baseline = np.mean([method_avg_delta[task_name][data_count]["Insert"], method_avg_delta[task_name][data_count]["Substitute"], method_avg_delta[task_name][data_count]["Translate"]])
        method_avg_delta[task_name][data_count]["Conventional Augmentation"] = baseline

# display(method_avg_delta)

pandas_form = {task_name: {} for task_name in results}
for task_name in pandas_form:
    for data_count in method_avg_delta[task_name]:
        for tta_method in ["Conventional Augmentation", "Paraphrase", "ICR"]:
            if data_count not in pandas_form[task_name]:
                pandas_form[task_name][data_count] = []

            pandas_form[task_name][data_count].append({
                "data_count": data_count,
                "tta_method": tta_method,
                "avg_delta": method_avg_delta[task_name][data_count][tta_method],
            })


fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE))
for i, task_name in enumerate(["BOSS_Sentiment", "BOSS_Toxicity", "AgNewsTweets"]):
    df = pd.concat([pd.DataFrame(pandas_form[task_name][data_count]) for data_count in pandas_form[task_name]])
    sns.lineplot(
        data=df,
        x="data_count",
        y="avg_delta",
        hue="tta_method",
        ax=axes[i],
        linewidth=LINE_WIDTH,
        marker="o",
        markersize=5)

    # set x label to Training Set Size
    axes[i].set_xlabel("Training Set Size")

    # set y label to Mean Absolute Accuracy Delta
    axes[i].set_ylabel("Accuracy Gain")

    # make the y axis percents that go to the hundreds place
    axes[i].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # standardize the y axis
    axes[i].set_ylim(-0.015, 0.1)

    title_text = {
        "BOSS_Sentiment": "Sentiment",
        "BOSS_Toxicity": "Toxicity",
        "AgNewsTweets": "News",
    }
    axes[i].set_title(title_text[task_name], fontsize=TITLE_FONT_SIZE)

    # se legend to the bottom left
    axes[i].legend(loc="lower right")

    # x axis ticks are five equally spaced ticks between the min and max of the x axis
    axes[i].set_xticks(np.linspace(df["data_count"].min(), df["data_count"].max(), 4))

    # remove leegnd in not middle plot
    if i != 1:
        axes[i].get_legend().remove()
    else:
        # center below plot with no frame
        axes[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.20), ncol=5, frameon=False, fontsize=14)

# add padding for labels
fig.subplots_adjust(wspace=WSPACE, hspace=WSPACE)

# plt.tight_layout()
if not os.path.exists("figures/"):
    os.makedirs("figures/")
fig.savefig("figures/method_analysis_data_ablation.png", bbox_inches="tight")

# OOD Analysis

In [ ]:
ood_sentiment_icr_data = None
ood_toxicity_icr_data = None
ood_tweets_icr_data = None
for split_name in tqdm(inference_logs.keys()):
    current_frame = inference_logs[split_name].to_pandas()
    current_frame["dataset"] = split_name

    if "Sentiment" in split_name and "ICR" in split_name and "ID" not in split_name and "BERT" in split_name and "Ablate" not in split_name:
        if ood_sentiment_icr_data is None:
            ood_sentiment_icr_data = current_frame
        else:
            ood_sentiment_icr_data = pd.concat([ood_sentiment_icr_data, current_frame])

    if "Toxicity" in split_name and "ICR" in split_name and "ID" not in split_name and "BERT" in split_name and "Ablate" not in split_name:
        if ood_toxicity_icr_data is None:
            ood_toxicity_icr_data = current_frame
        else:
            ood_toxicity_icr_data = pd.concat([ood_toxicity_icr_data, current_frame])

    if "Tweets" in split_name and "ICR" in split_name and "ID" not in split_name and "BERT" in split_name and "Ablate" not in split_name:
        if ood_tweets_icr_data is None:
            ood_tweets_icr_data = current_frame
        else:
            ood_tweets_icr_data = pd.concat([ood_tweets_icr_data, current_frame])


display(ood_sentiment_icr_data.value_counts("dataset"))
display(ood_toxicity_icr_data.value_counts("dataset"))
display(ood_tweets_icr_data.value_counts("dataset"))
assert len(ood_sentiment_icr_data.value_counts("dataset")) == 3
assert len(ood_toxicity_icr_data.value_counts("dataset")) == 3
assert len(ood_tweets_icr_data.value_counts("dataset")) == 1

## Aggregation Ablation Study

In [ ]:
def aggregate_predictions(predictions, num_predictions, use_test_input):
    try:
        ablation_preds = predictions[:num_predictions]
        if use_test_input:
            ablation_preds = ablation_preds.tolist() + [predictions[-1]]
        
        mean_distribution = np.mean(ablation_preds, axis=0) if len(ablation_preds) > 1 else ablation_preds[0]
        predicted_class = np.argmax(mean_distribution)
        return predicted_class
    except:
        return -1


perf_records = []
inference_ablation_splits = [split for split in inference_logs.keys() if "Ablate" not in split and "BERT" in split]
for split_name in tqdm(inference_ablation_splits):
    dataset = parse_task_name(split_name)
    distribution = parse_distribution(split_name)
    model = parse_model(split_name)
    method = parse_tta_method(split_name)

    current_frame = inference_logs[split_name].to_pandas()
    for use_source in [False, True]:
        for num_augmentations in range(1, 5):
            judgments = current_frame["tta_all_class_probs"].apply(lambda x: aggregate_predictions(x, num_augmentations, use_source))
            accuracy = classification_report(current_frame["label"], judgments, output_dict=True)["accuracy"]
            perf_records.append({
                "dataset": dataset,
                "distribution": distribution,
                "model": model,
                "method": method,
                "use_source": use_source,
                "num_augmentations": num_augmentations,
                "accuracy": accuracy,
                "delta": accuracy - no_tta_accuracies[split_name],
            })



In [ ]:
aggregation_ablation_frame = pd.DataFrame(perf_records)

# select where distribution != ID
aggregation_ablation_frame = aggregation_ablation_frame[aggregation_ablation_frame["distribution"] != "ID"]

# set method to TTA if not Paraphrase or ICR and LLM-TTA if Paraphrase or ICR
# aggregation_ablation_frame["method"] = aggregation_ablation_frame["method"].apply(lambda x: "TTA" if x not in ["Paraphrase", "ICR"] else "LLM-TTA")

# for tta_method in ["Conventional Augmentation", "Paraphrase", "ICR"]:
aggregation_ablation_frame["method"] = aggregation_ablation_frame["method"].apply(lambda x: "ICR" if "ICR" in x else "Paraphrase" if "Paraphrase" in x else "Conventional Augmentation")

# create three figures, one for Sentiment, Toxicity, and News
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE))
for index, task_name in enumerate(["Sentiment", "Toxicity", "News"]):
    # get the current task frame
    current_frame = aggregation_ablation_frame[aggregation_ablation_frame["dataset"] == task_name]

    # where use_source is True
    current_frame_source = current_frame[current_frame["use_source"] == True]

    # plot the current task frame
    sns.lineplot(
        data=current_frame,
        x="num_augmentations",
        y="delta",
        hue="method",
        # style="use_source",
        ax=axes[index],
        linewidth=LINE_WIDTH,
        ci=None,
        marker="o",
        markersize=5)

    # set x label to Number of Augmentations
    axes[index].set_xlabel("Number of Augmentations")

    # set y label to Accuracy
    axes[index].set_ylabel("Accuracy Gain")

    # make the y axis percents that go to the hundreds place
    axes[index].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # set title to Sentiment, Toxicity, or News
    axes[index].set_title(task_name, fontsize=TITLE_FONT_SIZE)

    # se legend to the bottom left
    axes[index].legend(loc="lower right")

    # standardize the y axis
    axes[i].set_ylim(-0.015, 0.15)

    # x axis ticks are five equally spaced ticks between the min and max of the x axis
    axes[index].set_xticks(np.linspace(current_frame["num_augmentations"].min(), current_frame["num_augmentations"].max(), 4))

    # remove leegnd in not middle plot
    if index != 1:
        axes[index].get_legend().remove()
    else:
        # center below plot with no frame
        axes[index].legend(loc="upper center", bbox_to_anchor=(0.5, -0.20), ncol=3, frameon=False, fontsize=14)
    
    # have y ticks to the tenths place
    axes[index].yaxis.set_major_formatter(lambda x, pos: f"{x:.1%}")

# add padding for labels
fig.subplots_adjust(wspace=WSPACE + 0.05, hspace=WSPACE)

fig.savefig("figures/method_analysis_aggrgeation_ablation.png", bbox_inches="tight")

In [ ]:
# don't crop the display frame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

aggregation_ablation_frame = pd.DataFrame(perf_records)
for dataset in ["Sentiment", "Toxicity", "News"]:
    dataset_frame = aggregation_ablation_frame[aggregation_ablation_frame["dataset"] == dataset]
    dataset_frame["distribution"] = dataset_frame["distribution"].apply(lambda x: "ID" if "ID" in x else "OOD")
    display(dataset_frame.groupby(["dataset", "distribution", "model", "method", "use_source", "num_augmentations"]).mean().T)

## Does TTA Effect Some Classes More Than Others?

In [ ]:
# for each dataset, get the percent of examples that are unchanged vs new
sentiment_outcomes = ood_sentiment_icr_data["outcome"]
sentiment_percents = sentiment_outcomes.value_counts(normalize=True)
new_sentiment_percents = 100 * sentiment_percents[sentiment_percents.index == "New Correct"].values[0] + sentiment_percents[sentiment_percents.index == "New Mistake"].values[0]
print(f"Sentiment: {new_sentiment_percents:.2f}% of examples are new predictions")

toxicity_outcomes = ood_toxicity_icr_data["outcome"]
toxicity_percents = toxicity_outcomes.value_counts(normalize=True)
new_toxicity_percents = 100 * toxicity_percents[toxicity_percents.index == "New Correct"].values[0] + toxicity_percents[toxicity_percents.index == "New Mistake"].values[0]
print(f"Toxicity: {new_toxicity_percents:.2f}% of examples are new predictions")

agt_outcomes = ood_tweets_icr_data["outcome"]
agt_percents = agt_outcomes.value_counts(normalize=True)
new_agt_percents = 100 * agt_percents[agt_percents.index == "New Correct"].values[0] + agt_percents[agt_percents.index == "New Mistake"].values[0]
print(f"AGT: {new_agt_percents:.2f}% of examples are new predictions")

In [ ]:
sentiment_icr_outcome_percents = ood_sentiment_icr_data[["outcome", "label"]].sort_values("outcome").value_counts(["outcome", "label"], normalize=True).sort_index()
sentiment_icr_outcome_percents = sentiment_icr_outcome_percents[sentiment_icr_outcome_percents.index.get_level_values("outcome").str.contains("New")]
print("Sentiment ICR")
display(sentiment_icr_outcome_percents)

toxicity_icr_outcome_percents = ood_toxicity_icr_data[["outcome", "label"]].sort_values("outcome").value_counts(["outcome", "label"], normalize=True).sort_index()
toxicity_icr_outcome_percents = toxicity_icr_outcome_percents[toxicity_icr_outcome_percents.index.get_level_values("outcome").str.contains("New")]
print("Toxicity ICR")
display(toxicity_icr_outcome_percents)

tweets_icr_outcome_percents = ood_tweets_icr_data[["outcome", "label"]].sort_values("outcome").value_counts(["outcome", "label"], normalize=True).sort_index()
tweets_icr_outcome_percents = tweets_icr_outcome_percents[tweets_icr_outcome_percents.index.get_level_values("outcome").str.contains("New")]
print("Tweets ICR")
display(tweets_icr_outcome_percents)

In [ ]:
plt.clf()
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE))

sentiment_labels = {
    0: "Negative",
    1: "Positive",
    2: "Neutral",
}
sns.barplot(ax=axes[0], 
            x=sentiment_icr_outcome_percents.index.get_level_values("label").map(sentiment_labels), 
            y=sentiment_icr_outcome_percents.values,
            hue=sentiment_icr_outcome_percents.index.get_level_values("outcome"),
)

toxicity_labels = {
    0: "Non-Toxic",
    1: "Toxic",
}
sns.barplot(ax=axes[1],
            x=toxicity_icr_outcome_percents.index.get_level_values("label").map(toxicity_labels),
            y=toxicity_icr_outcome_percents.values,
            hue=toxicity_icr_outcome_percents.index.get_level_values("outcome"),
)

agt_labels = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
}
sns.barplot(ax=axes[2],
            x=tweets_icr_outcome_percents.index.get_level_values("label").map(agt_labels),
            y=tweets_icr_outcome_percents.values,
            hue=tweets_icr_outcome_percents.index.get_level_values("outcome"),
)

for i in range(3):
    # make y axis percents
    axes[i].yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

    # standardize between 0 and 0.1
    axes[i].set_ylim(0, 0.09)

    # set y label to percent of overall outcomes
    axes[i].set_ylabel("Percent of Outcomes")

    # set x label to Class
    axes[i].set_xlabel("Class")

    # have fewer ticks on the y axis
    axes[i].locator_params(axis="y", nbins=5)

    # set titles
    title_text = {
        0: "Sentiment",
        1: "Toxicity",
        2: "News",
    }
    axes[i].set_title(title_text[i], fontsize=TITLE_FONT_SIZE)

    # have a single legend which is centered below the plot
    if i == 1:
        axes[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.20), ncol=3, frameon=False, fontsize=14)
    else:
        axes[i].get_legend().remove()

# add more horizental spacing for y labels
fig.subplots_adjust(wspace=WSPACE)
fig.savefig("figures/method_analysis_class_analysis.png", bbox_inches="tight")
fig.show()

# Selective Augmentation

In [ ]:
# three figures on single line
fig, axes = plt.subplots(1, 2, figsize=(3 * FIG_SIZE, FIG_SIZE))
plottng_datas = [("Sentiment", ood_sentiment_icr_data), ("Toxicity", ood_toxicity_icr_data), ("News", ood_tweets_icr_data)]
change_int_mapping = {
    "Unchanged": 0,
    "New": 1,
}
outcome_int_mapping = {
    "Unchanged Correct": 0,
    "Unfixed Mistake": 1,
    "New Correct": 2,
    "New Mistake": 3,
}

change_correlations = []
outcome_correlations = []
for index, (title, icr_frame) in enumerate(plottng_datas):
    icr_frame["max_softmax"] = icr_frame["tta_mean_class_probs"].apply(lambda p: max(p))
    plotting_frame = icr_frame[["max_softmax", "original_prediction_entropy", "tta_prediction_entropy", "outcome"]].sort_values("outcome")
    plotting_frame["Change"] = plotting_frame["outcome"].apply(lambda x: 0 if x == "Unchanged Correct" or x == "Unfixed Mistake" else 1)

    correlation_metric = "original_prediction_entropy"
    pbc = pointbiserialr(plotting_frame[correlation_metric], plotting_frame["Change"])
    change_correlations.append([pbc[0]])

    set_outcome_correlations = []
    for outcome in outcome_int_mapping:
        one_hot_outcomes = plotting_frame["outcome"].apply(lambda x: 1 if x == outcome else 0)
        pbc = pointbiserialr(plotting_frame[correlation_metric], one_hot_outcomes)
        set_outcome_correlations.append(pbc[0])
    
    outcome_correlations.append(set_outcome_correlations)

sns.heatmap(change_correlations,
            annot=True,
            vmin=-1,
            vmax=1,
            ax=axes[0],
            cmap="coolwarm",
            xticklabels=["Prediction Changed"],
            yticklabels=["Sentiment", "Toxicity", "News"])

sns.heatmap(outcome_correlations,
            annot=True,
            vmin=-1,
            vmax=1,
            ax=axes[1],
            cmap="coolwarm",
            xticklabels=["Unchanged Correct", "Unfixed Mistake", "New Correct", "New Mistake"],
            yticklabels=["Sentiment", "Toxicity", "News"])

plt.xticks(rotation=20, ha="right")
fig.subplots_adjust(wspace=WSPACE / 2)
fig.savefig("figures/method_analysis_entropy_correlations.png", bbox_inches="tight")

In [ ]:
# three figures on single line
fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE))
plottng_datas = [("Sentiment", ood_sentiment_icr_data), ("Toxicity", ood_toxicity_icr_data), ("News", ood_tweets_icr_data)]

correlations = []

for index, (title, icr_frame) in tqdm(enumerate(plottng_datas)):
    axes[index].set_title(title, fontsize=TITLE_FONT_SIZE)
    icr_frame["max_softmax"] = icr_frame["tta_mean_class_probs"].apply(lambda p: max(p))
    plotting_frame = icr_frame[["max_softmax", "original_prediction_entropy", "tta_prediction_entropy", "outcome"]].sort_values("outcome")
    plotting_frame["Prediction"] = plotting_frame["outcome"].apply(lambda x: "Unchanged Prediction" if x == "Unchanged Correct" or x == "Unfixed Mistake" else "New Prediction")

    # create kernal density estimate plot for original prediction entropy by outcome
    sns.kdeplot(
        data=plotting_frame,
        x="original_prediction_entropy",
        hue="Prediction",
        fill=True,
        ax=axes[index],
        linewidth=LINE_WIDTH,
        # alpha=0.5,
        common_norm=False,
    )

    # log y and x axis
    # axes[index].set_yscale("log")
    axes[index].set_xscale("log")

    # move legend to the top left
    if index == 1:
        axes[index].legend(loc="upper center", labels=["Unchanged", "New"], bbox_to_anchor=(0.5, -0.20), ncol=2, frameon=False, fontsize=14)
    else:
        axes[index].get_legend().remove()

fig.subplots_adjust(wspace=WSPACE)
fig.savefig("figures/method_analysis_entropy_distributions.png", bbox_inches="tight")

## Calculate Optimal ID Entropy Thresholds

In [ ]:
def should_augment_entropy(threshold, row):
    return row["original_prediction_entropy"] >= threshold


def get_entropy_threshold_accuracy(threshold, inference_logs_frame):
    threshold_judgments = inference_logs_frame.apply(lambda row: row["tta_predicted_class"] if should_augment_entropy(threshold, row) else row["original_predicted_class"], axis=1)
    report = classification_report(inference_logs_frame["label"], threshold_judgments, digits=4, output_dict=True, zero_division=0)
    llm_call_count = inference_logs_frame.apply(lambda row: should_augment_entropy(threshold, row), axis=1).sum()
    llm_call_rate = llm_call_count / len(inference_logs_frame)
    return report["accuracy"], llm_call_rate


def should_augment_softmax(threshold, row):
    try:
        return row["tta_all_class_probs"][-1].max() < threshold
    except:
        return False


def get_max_softmax_threshold_accuracy(threshold, inference_logs_frame):
    threshold_judgments = inference_logs_frame.apply(lambda row: row["tta_predicted_class"] if should_augment_softmax(threshold, row) else row["original_predicted_class"], axis=1)
    report = classification_report(inference_logs_frame["label"], threshold_judgments, digits=4, output_dict=True, zero_division=0)
    llm_call_count = (inference_logs_frame["original_prediction_entropy"] >= threshold).sum()
    llm_call_rate = llm_call_count / len(inference_logs_frame)
    return report["accuracy"], llm_call_rate

thresholds = np.arange(0, 1.2, 0.0001)
print(f"Number of thresholds: {len(thresholds)}")
thresholds

## OOD Entropy Threshold Analysis 

In [ ]:
# 1. Calculate all the tresholds for each OOD split for BERT ICR. 
# 2. Get the manual threshold for each OOD split for BERT ICR at different augmentation rates.

thresholds_dict = {}
thresholds_path = f"data/thresholds_dict_{len(thresholds)}.json"
if not os.path.exists(thresholds_path):
    for ood_icr_data in [ood_sentiment_icr_data, ood_toxicity_icr_data, ood_tweets_icr_data]:
        for split in ood_icr_data["dataset"].unique():
            print(split)
            thresholds_dict[split] = {}
            split_frame = ood_icr_data[ood_icr_data["dataset"] == split]
            original_accuracy = classification_report(split_frame["label"], split_frame["original_predicted_class"], output_dict=True)["accuracy"]
            
            for threshold in tqdm(thresholds):
                accuracy, llm_call_rate = get_entropy_threshold_accuracy(threshold, split_frame)
                thresholds_dict[split][threshold] = {
                    "accuracy": accuracy,
                    "accuracy_delta": accuracy - original_accuracy,
                    "llm_call_rate": llm_call_rate,
                }
    json.dump(thresholds_dict, open(thresholds_path, "w"), indent=4)
else:
    with open("data/thresholds_dict.json", "r") as f:
        thresholds_dict = json.load(f)

print(json.dumps(thresholds_dict, indent=4))

In [ ]:
# create a subplot for each dataset in the thresholds dict with three on each row
fig, axes = plt.subplots(3, 3, figsize=(3 * FIG_SIZE, 3 * FIG_SIZE))

# use the first row for sentiment, second for toxicity, and third for agt
task_keys = [
    [key for key in thresholds_dict.keys() if "Sentiment" in key],
    [key for key in thresholds_dict.keys() if "Toxicity" in key],
    [key for key in thresholds_dict.keys() if "Tweets" in key],
]
for row_index, task_splits in enumerate(task_keys):
    for col_index in range(len(task_splits)):
        split_name = task_splits[col_index]
        thresholds_split_frame = pd.DataFrame(thresholds_dict[split_name]).T.reset_index().sort_values("llm_call_rate")
        sns.lineplot(
            ax=axes[row_index, col_index],
            data=thresholds_split_frame,
            x="llm_call_rate",
            y="accuracy_delta",
            linewidth=LINE_WIDTH)

        row_titles = {
            0: "Sentiment",
            1: "Toxicity",
            2: "News",
        }
        shift_name = split_name.split("_")[-3]
        axes[row_index, col_index].set_title(f"{row_titles[row_index]}: {shift_name}", fontsize=TITLE_FONT_SIZE)

for col_index in range(3):
    for row_index in range(3):
        axes[row_index, col_index].set_ylabel("Accuracy Delta")
        axes[row_index, col_index].set_xlabel("Augmentation Rate")

        # set x and y axis to percents
        axes[row_index, col_index].xaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")

        # multiple delta by 100 to get percent
        axes[row_index, col_index].yaxis.set_major_formatter(lambda x, pos: f"{round(x * 100, 2)}")

        # set y axis between -0.1 and 0.1
        # axes[row_index, col_index].set_ylim(-0.01, 0.15)

        # have five ticks on the y axis
        axes[row_index, col_index].locator_params(axis="y", nbins=8)

        # delete last two plots on the final row
        if row_index == 2 and col_index > 0:
            axes[row_index, col_index].remove()

# add padding for labels
fig.subplots_adjust(wspace=WSPACE, hspace=WSPACE + 0.25)

# save figure
fig.savefig("figures/method_analysis_all_entropy_thresholds.png", bbox_inches="tight")

In [ ]:
aggregated_sentiment_thresholds = {}
for split in thresholds_dict:
    task = "Sentiment" if "Sentiment" in split else "Toxicity" if "Toxicity" in split else "News"
    if task not in aggregated_sentiment_thresholds:
        aggregated_sentiment_thresholds[task] = {}

    for threshold in thresholds_dict[split]:
        if threshold not in aggregated_sentiment_thresholds:
            aggregated_sentiment_thresholds[task][threshold] = {
                "accuracy_delta": 0,
                "llm_call_rate": 0,
            }

        aggregated_sentiment_thresholds[task][threshold]["accuracy_delta"] += thresholds_dict[split][threshold]["accuracy_delta"]
        aggregated_sentiment_thresholds[task][threshold]["llm_call_rate"] += thresholds_dict[split][threshold]["llm_call_rate"]

# divide each accuracy delta by the number of splits to get the average
for task in aggregated_sentiment_thresholds:
    for threshold in aggregated_sentiment_thresholds[task]:
        aggregated_sentiment_thresholds[task][threshold]["accuracy_delta"] /= 3
        # aggregated_sentiment_thresholds[task][threshold]["llm_call_rate"] /= 3

fig, axes = plt.subplots(1, 3, figsize=(3 * FIG_SIZE, FIG_SIZE))
for col_index, task in enumerate(aggregated_sentiment_thresholds):
    sns.lineplot(
        ax=axes[col_index],
        data=pd.DataFrame(aggregated_sentiment_thresholds[task]).T.reset_index().sort_values("llm_call_rate"),
        x="llm_call_rate",
        y="accuracy_delta",
        linewidth=LINE_WIDTH)
    

    axes[col_index].set_title(task, fontsize=TITLE_FONT_SIZE)

    # add padding for labels
    axes[col_index].set_ylabel("Accuracy Delta")
    axes[col_index].set_xlabel("Augmentation Rate")

    # set x and y axis to percents
    axes[col_index].xaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")
    axes[col_index].yaxis.set_major_formatter(lambda x, pos: f"{round(x * 100, 2)}")

    # set y value between 0 and 0.1
    # axes[col_index].set_ylim(-0.01, 0.08)

    # have few ticks on the x axis
    axes[col_index].locator_params(axis="x", nbins=5)
    axes[col_index].locator_params(axis="y", nbins=5)

fig.subplots_adjust(wspace=WSPACE, hspace=0.3)
fig.savefig("figures/method_analysis_aggregated_entropy_thresholds.png", bbox_inches="tight")

### Selective Augmentation

In [ ]:
def set_tta_preds(row):
    if row["generations"] is not None and len(row["generations"]) > 0:
        return row["generations"][:5]
    
    if row["tta_all_class_probs"] is None:
        return None

    all_probs = row["tta_all_class_probs"][:5]
    arg_maxes = [prob_dist.argmax() for prob_dist in all_probs]
    return arg_maxes

def is_entropy_split(split_name):
    if "BERT" not in split_name:
        return False
    if "Ablate" in split_name:
        return False
    if "ID" in split_name:
        return False

    return "Paraphrase" in split_name or "ICR" in split_name

In [ ]:
optimal_entropy_thresholds = {}
optimal_softmax_thresholds = {}
thresholds = np.arange(0, 1.2, 0.0005)
SAMPLE_SIZE = 250
print(f"Number of thresholds: {len(thresholds)}")

for split in [dataset for dataset in inference_logs if is_entropy_split(dataset)]:
    print(split)
    best_entropy_threshold = None
    best_softmax_threshold = None
    split_frame = inference_logs[split].to_pandas()
    sample_frame = None
    unique_predicted_classes = [class_label for class_label in split_frame["tta_predicted_class"].unique() if class_label != -1] 
    for class_prediction in unique_predicted_classes:
        sample_size = SAMPLE_SIZE // len(unique_predicted_classes)
        class_sample_frame = split_frame[split_frame["tta_predicted_class"] == class_prediction].sample(sample_size, random_state=42)
        if sample_frame is None:
            sample_frame = class_sample_frame
        else:
            sample_frame = pd.concat([sample_frame, class_sample_frame])
    
    threshold_performances = []
    for threshold in tqdm(thresholds):
        accuracy, llm_call_rate = get_entropy_threshold_accuracy(threshold, sample_frame)
        beta = 1/500
        rate_term = 1 - llm_call_rate
        threshold_score = (1 + beta ** 2) * ((accuracy * rate_term) / ((beta ** 2) * accuracy + rate_term))
        threshold_perf = {
            "threshold": threshold,
            "accuracy": accuracy,
            "score": threshold_score,
            "llm_call_rate": f"{llm_call_rate:.2f}%",
        }
        threshold_performances.append(threshold_perf)

        # if best_entropy_threshold is None or accuracy > best_entropy_threshold["accuracy"]:
        if best_entropy_threshold is None or threshold_score > best_entropy_threshold["score"]:
            best_entropy_threshold = threshold_perf

    pd.DataFrame(threshold_performances).to_csv(f"data/threshold_performances_{split}.csv", index=False)
    optimal_entropy_thresholds[split] = best_entropy_threshold
    print(f"Best Entropy Threshold: {best_entropy_threshold}")

# print(json.dumps(optimal_entropy_thresholds, indent=4))
# print(json.dumps(optimal_softmax_thresholds, indent=4))


In [ ]:
perf_records = []

# display(id_logs)
# display(split_names)
for split_name in tqdm(optimal_entropy_thresholds):
    split_logs = inference_logs[split_name].to_pandas()

    perf_records.append({
        "split": split_name,
        "tta": "None",
        "accuracy": classification_report(split_logs["label"], split_logs["original_predicted_class"], digits=4, zero_division=0, output_dict=True)["accuracy"],
    })

    optimal_entropy_threshold = optimal_entropy_thresholds[split_name]["threshold"]
    accuracy = get_entropy_threshold_accuracy(optimal_entropy_threshold, split_logs)[0]
    perf_records.append({
        "split": split_name,
        "tta": "entropy-based",
        "accuracy": accuracy,
        "augmentation_rate": split_logs.apply(lambda row: should_augment_entropy(optimal_entropy_threshold, row), axis=1).sum() / len(split_logs),
    })


In [ ]:
results_frame = pd.DataFrame(perf_records)
results_frame["Dataset"] = results_frame["split"].apply(lambda s: s.split("_")[-4])
results_frame["Distribution"] = results_frame["split"].apply(lambda s: s.split("_")[-3])
results_frame["Model"] = results_frame["split"].apply(lambda s: s.split("_")[-2])
results_frame["TTA Method"] = results_frame["split"].apply(lambda s: s.split("_")[-1])
results_frame["Baseline Delta"] = results_frame.apply(lambda row: row["accuracy"] - results_frame[(results_frame["split"] == row["split"]) & (results_frame["tta"] == "None")]["accuracy"].values[0], axis=1)
results_frame.drop(columns=["split"], inplace=True)
results_frame.rename(columns={"tta": "Selective Method", "accuracy": "Accuracy", "augmentation_rate": "Augmentation Rate"}, inplace=True)

aggregated_results = results_frame.groupby(["Dataset", "Distribution", "Model", "TTA Method", "Selective Method"]).mean().round(4) * 100
aggregated_results = aggregated_results.sort_values(by=["Dataset", "Distribution", "Model", "TTA Method", "Accuracy"], ascending=False)

print("Overall Results")
# display(aggregated_results)

# Average each TTA Method and Selective Method over distributions
results_frame["ID"] = results_frame["Distribution"].apply(lambda d: "ID" in d)
results_frame.drop(columns=["Distribution", "Accuracy"], inplace=True)
results_frame = results_frame[["Dataset", "ID", "Model", "TTA Method", "Selective Method", "Baseline Delta", "Augmentation Rate"]]
aggregated_results = results_frame.groupby(["Dataset", "ID", "Model", "TTA Method", "Selective Method"]).mean().round(4) * 100
aggregated_results = aggregated_results.sort_values(by=["Dataset", "ID", "Model", "TTA Method", "Baseline Delta"], ascending=False)
print("Average Results")
for dataset in ["Sentiment", "Toxicity", "AgNewsTweets"]:
    for tta_method in ["Paraphrase", "ICR"]:
    # for tta_method in ["ICR"]:
        print(f"Dataset: {dataset}, TTA Method: {tta_method}")
        # display(aggregated_results.loc[dataset, :, :, tta_method, :])
        # only show entropy-based
        display(aggregated_results.loc[dataset, :, :, tta_method, "entropy-based"])